![](ubc_header.png)

# Climate Friendly Food System (CFFS) Labelling Pilot
***

## Objective

Implement the Climate-Friendly and Just Food System (CFFS &Just) definition at the UBC Campus by producing the weighted metric that informs the choice of icon for each menu item.

## Data Source

**Recipes Data**: extracted from UBC culinary services management platform Optimum Control
    
    Items.xml: Raw ingredient Items
    Ingredients.xml: Ingredients/Preps that goes into prep/products recipes
    Preps.xml: Preps that go into final product recipes
    Products.xml: Final Recipes/Menu Items
    Conversions.xml: Unit Conversion Information

**cool_food_ghg.csv**: extracted from the [Cool Food Calculator](https://coolfood.org/pledge/). For this project, we use emission factors for the active total supply chain emissions in the North America region. The food category is at the detailed level for higher results accuracy.

## Assumptions

* The same GHG emission factor will be assigned to different forms (puree, sliced, chopped etc) of the same raw ingredient

* We assume the GHG factors for different varieties of one ingredient are the same (i.e Red/Yellow Onion)

* The loss/add of ingredient weight during the cooking process will be ignored

* The amount of unusable part and wastes during the ingredients processing process will be ignored 

* Ignore GHG emissions from the cooking process

* Assume the GHG emission factor for water is zero, and ignore the water use in the cooking process

* We exclude the GHG emission from sauces/dressing that has no dominant ingredients

## Import Libraries

In [1]:
#!pip3 install pdpipe

In [2]:
import numpy as np
import pandas as pd
import pdpipe as pdp
import matplotlib.pyplot as plt
import glob
import os

import csv
from itertools import islice
from decimal import Decimal

In [3]:
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse

In [4]:
import openpyxl

In [5]:
pd.set_option("display.max_rows", None, "display.max_columns", None)

## Load Data

#### Data Path

In [6]:
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "oc", "*.oc"))

In [7]:
filepath_list

['/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1418.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1423.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1417.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1422.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1416.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1421.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1415.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1414.oc',
 '/Users/silvia/cffs-label/data analysis/data/oc/IPR_Export_04272021_1424.oc']

#### Load Items List

In [8]:
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    path = filepath + '/items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for item in xtree.iterfind('Item'):
        ItemId.append(item.attrib['id'])
        Description.append(item.findtext('Description'))
        CaseQty.append(item.findtext('CaseQty'))
        CaseUOM.append(item.findtext('CaseUOM'))
        PakQty.append(item.findtext('PakQty'))
        PakUOM.append(item.findtext('PakUOM'))
        InventoryGroup.append(item.findtext('InventoryGroup'))

        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    ).drop_duplicates()

Items.reset_index(drop=True, inplace=True)

In [9]:
Items.head()

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-46620,ICE TEA GRN GOLD PEAK,12.000,ea,1.000,ea,BEVERAGE
1,I-46621,ICE TEA LEMON GOLD PEAK,12.000,ea,1.000,ea,BEVERAGE
2,I-46622,ICE TEA UNSWT GOLD PEAK,12.000,ea,1.000,ea,BEVERAGE
3,I-20202,JAVA JACKET GENERIC HOLIDAY WR,1300.000,CT,1.000,CT,DISPOSABLES
4,I-43254,JUICE 5 ALIVE 355ML,12.000,can,1.000,355ML,BEVERAGE


In [10]:
Items.shape

(202, 7)

In [11]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [12]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

#### Load Ingredient List

In [13]:
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Ingredient'):
        IngredientId.append(x.attrib['ingredient'])
        Conversion.append(x.attrib['conversion'])
        InvFactor.append(x.attrib['invFactor'])
        Qty.append(x.attrib['qty'])
        Recipe.append(x.attrib['recipe'])
        Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()

Ingredients.reset_index(drop=True, inplace=True)

In [14]:
Ingredients.head()

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,I-2058,1.000,Kg,1.00000000,0.0667,P-10395
1,I-2061,500.000,g,0.00100000,33.3333,P-10395
2,I-7454,0.250,Kg,1.00000000,0.0167,P-10395
3,P-5545,10.000,L,1.00000000,0.6667,P-10395
4,I-3276,250.000,ml,0.00100000,0.1250,P-25843


In [15]:
Ingredients.shape

(596, 6)

In [16]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [17]:
path = os.path.join(os.getcwd(), "data", "processed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

#### Load Preps List

In [18]:
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Prep'):
        PrepId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        PakQty.append(x.findtext('PakQty'))
        PakUOM.append(x.findtext('PakUOM'))
        InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()

Preps.reset_index(drop=True, inplace=True)

In [19]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-5545,PREP|Bechamel,10.000,L,PREP
1,P-30411,PREP|Cooked Spelt,2.000,Kg,
2,P-5962,PREP|Pesto,20.000,L,PREP
3,P-10395,PREP|Sauce Cheese,15.000,L,
4,P-30409,PREP|Tri Colour Rotini,2.000,Kg,


In [20]:
Preps.shape

(27, 5)

In [21]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [22]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

#### Load Product List

In [23]:
ProdId = []
Description = []

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Prod'):
        ProdId.append(x.attrib['id'])
        Description.append(x.findtext('Description'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description}).drop_duplicates()

Products.reset_index(drop=True, inplace=True)

In [24]:
Products.head()

,ProdId,Description
0,R-54100,BEV|Eggnog Chai
1,R-50814,BEV|Water|Dasani|591ml
2,R-25866,"MC- Salad Watermelon, tomato"
3,R-30415,Salad- Spelt & Arugula
4,R-30414,SALAD|Antipasto Pasta


In [25]:
Products.shape

(257, 2)

In [26]:
Products.dtypes

ProdId         object
Description    object
dtype: object

In [27]:
path = os.path.join(os.getcwd(), "data", "processed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

#### Load Conversion List

In [28]:
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
    for x in xtree.iterfind('Conversion'):
        ConversionId.append(x.attrib['id'])
        Multiplier.append(x.attrib['multiplier'])
        ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
        ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
        ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
        ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [29]:
Conversions.head()

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L


In [30]:
Conversions.shape

(98, 6)

In [31]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [32]:
path = os.path.join(os.getcwd(), "data", "processed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

#### Load Cool Food Calculator Emission Factors

In [33]:
cfc_filename = os.path.join(os.getcwd(), "data", "raw", "cool_food_ghg.csv")
cfc = pd.read_csv(cfc_filename)

In [34]:
cfc.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [35]:
cfc.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [36]:
cfc.shape

(61, 3)

## Data Cleaning

#### Unit Conversion

In [37]:
Preps['StdQty'] = np.nan
Preps['StdUom'] = np.nan

for index in Preps.index:
    if Preps.loc[index,'PakUOM'] == 'Kg':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'lb':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*453.592
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'oz':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*28.3495
        Preps.loc[index,'StdUom'] = 'g'
    elif Preps.loc[index,'PakUOM'] == 'L':
        Preps.loc[index,'StdQty'] = float(Preps.loc[index,'PakQty'])*1000
        Preps.loc[index,'StdUom'] = 'ml'
    else: 
        Preps.loc[index,'StdQty'] = Preps.loc[index,'PakQty']
        Preps.loc[index,'StdUom'] = Preps.loc[index,'PakUOM']
        

In [38]:
Preps.head()

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom
0,P-5545,PREP|Bechamel,10.000,L,PREP,10000,ml
1,P-30411,PREP|Cooked Spelt,2.000,Kg,,2000,g
2,P-5962,PREP|Pesto,20.000,L,PREP,20000,ml
3,P-10395,PREP|Sauce Cheese,15.000,L,,15000,ml
4,P-30409,PREP|Tri Colour Rotini,2.000,Kg,,2000,g


In [39]:
path = os.path.join(os.getcwd(), "data", "processed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

## Data Analysis

#### Add New Items to Database

In [40]:
# Load current Items List with assigned Cool Food Category ID
Items_Assigned = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv"))
Items_Assigned.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [41]:
col_names = list(Items.columns.values)

In [42]:
New_Items_List = []

for index, row in Items.iterrows():
    ItemId = Items.loc[index,'ItemId']
    if ItemId not in Items_Assigned['ItemId'].values:
        Dict = {}
        Dict.update(dict(row))
        New_Items_List.append(Dict)

New_Items = pd.DataFrame(New_Items_List, columns = col_names)

In [43]:
#Items_Assigned['ItemId'].values

In [44]:
New_Items.insert(1, "CategoryID", '')

In [45]:
New_Items

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [46]:
from datetime import datetime
path = os.path.join(os.getcwd(), "data", "new items", str(datetime.date(datetime.now()))+"_New_Items.csv")
New_Items.to_csv(path, index = False, header = True)

In [47]:
New_Items_Added = pd.read_csv(os.path.join(os.getcwd(), "data", "new items added", "New_Items_Added_2.csv"))
New_Items_Added

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-2333,3,PROSCIUTTO SLICED SAN DANIELE,1,Kg,1,Kg,DELI & PREPARED MEAT
1,I-3700,61,YEAST BAKERS FRSH,24,lb,1,lb,FOOD - GROCERY


In [48]:
frames = [Items_Assigned, New_Items_Added]
Items_Assigned_Updated = pd.concat(frames).reset_index(drop=True, inplace=False).drop_duplicates()
Items_Assigned_Updated.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [49]:
Items_Assigned_Updated[['CategoryID']] = Items_Assigned_Updated[['CategoryID']].apply(pd.to_numeric)

In [50]:
path = os.path.join(os.getcwd(), "data", "processed", "Items_List_Assigned.csv")
Items_Assigned_Updated.to_csv(path, index = False, header = True)

#### Mapping Items to GHG Factors

In [51]:
df1 = pd.DataFrame(Items_Assigned_Updated)
df2 = pd.DataFrame(cfc)

In [52]:
df1.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.0,Kg,MEAT
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.0,CT,MEAT
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.0,Kg,MEAT
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.0,Kg,MEAT
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.0,g,MEAT


In [53]:
df1.dtypes

ItemId             object
CategoryID        float64
Description        object
CaseQty           float64
CaseUOM            object
PakQty            float64
PakUOM             object
InventoryGroup     object
dtype: object

In [54]:
df2.head()

,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,1,beef & buffalo meat,41.3463
1,2,lamb/mutton & goat meat,41.6211
2,3,pork (pig meat),9.8315
3,4,"poultry (chicken, turkey)",4.3996
4,5,butter,11.4316


In [55]:
df2.dtypes

Category ID                                                 int64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [56]:
mapping = pd.merge(df1, df2.loc[:,['Category ID','Food Category','Active Total Supply Chain Emissions (kg CO2 / kg food)']], 
                  how = 'left',
                  left_on = 'CategoryID', 
                  right_on = 'Category ID')

In [57]:
for index in mapping.index:
    if np.isnan(mapping.loc[index,'Category ID']):
        mapping.loc[index,'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = 0

##### Manully Adjust GHG Factor for Some Items

In [58]:
Manual_GHG = pd.read_csv(os.path.join(os.getcwd(), "data", "processed", "Manual_Adjust_GHGe.csv"))
Manual_GHG

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-3223,59,COCONUT MILK 17/19% MILK FAT,6.00,LG CAN,2.84,L,FOOD - GROCERY,59,manually adjusted,3.50000
1,I-3370,59,MUSTARD DIJON HOT,2.00,each,4625.00,ml,FOOD - GROCERY,59,manually adjusted,2.32600
2,I-3368,59,MUSTARD DIJON GRAINY OLD FASH,2.00,pail,4625.00,ml,FOOD - GROCERY,59,manually adjusted,2.32600
3,I-2897,59,MUSTARD DIJON GRAINY FLEUR,4.00,Kg,1.00,Kg,FOOD - GROCERY,59,manually adjusted,2.32600
4,I-3365,59,MUSTARD BIG YELLOW PLAS,4.00,JUG,2.84,L,FOOD - GROCERY,59,manually adjusted,1.76000
5,I-3352,59,MAYONNAISE PAIL TFC 4L,2.00,each,4.00,L,FOOD - GROCERY,59,manually adjusted,3.55000
6,I-3348,59,MAYONNAISE 1/2 FAT TFC,2.00,4L,4.00,L,FOOD - GROCERY,59,manually adjusted,1.22000
7,I-3336,59,KETCHUP BIG RED PLAS,6.00,LG CAN,2.84,L,FOOD - GROCERY,59,manually adjusted,2.01000
8,I-3284,59,EXTRACT VANILLA PURE,1.00,500ML,1.00,500ML,FOOD - GROCERY,59,manually adjusted,0.00000
9,I-52090,1,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.00,CT,MEAT,1,beef & buffalo meat,31.13750


In [59]:
for index, row in Manual_GHG.iterrows():
    itemId = Manual_GHG.loc[index, 'ItemId']
    ghge = Manual_GHG.loc[index, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
    mapping.loc[mapping['ItemId'] == itemId, 'Active Total Supply Chain Emissions (kg CO2 / kg food)'] = ghge

In [60]:
mapping#.head()

,ItemId,CategoryID,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup,Category ID,Food Category,Active Total Supply Chain Emissions (kg CO2 / kg food)
0,I-57545,1.0,CHUCK FLAT BONELESS FZN,3.30,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
1,I-52090,1.0,BURGER BEEF & MUSHROOM HALAL,1.00,cs,48.000,CT,MEAT,1.0,beef & buffalo meat,31.13750
2,I-10869,1.0,BEEF STIRFRY COV FR,5.00,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
3,I-7064,1.0,BEEF OUTSIDE FLAT AAA,1.00,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
4,I-37005,1.0,BEEF MEATBALLS,4.54,Kg,1000.000,g,MEAT,1.0,beef & buffalo meat,41.34630
5,I-37002,1.0,BEEF INSIDE ROUND SHAVED,9.00,Kg,1000.000,g,MEAT,1.0,beef & buffalo meat,41.34630
6,I-3876,1.0,BEEF CHUCK GROUND AAA,1.00,Kg,1.000,Kg,MEAT,1.0,beef & buffalo meat,41.34630
7,I-13303,3.0,SAUSAGE PATTIES PORK,5.00,Kg,20.000,ea,MEAT,3.0,pork (pig meat),9.83150
8,I-11922,3.0,SAUSAGE MEAT CHORIZO FRZN,5.00,CT,2.000,Kg,MEAT,3.0,pork (pig meat),9.83150
9,I-50575,3.0,SAUSAGE ITAL CRUMBLE CKD FRZ,2.00,bag,2.270,Kg,MEAT,3.0,pork (pig meat),9.83150


In [61]:
mapping.shape

(555, 11)

In [62]:
mapping.dtypes

ItemId                                                     object
CategoryID                                                float64
Description                                                object
CaseQty                                                   float64
CaseUOM                                                    object
PakQty                                                    float64
PakUOM                                                     object
InventoryGroup                                             object
Category ID                                               float64
Food Category                                              object
Active Total Supply Chain Emissions (kg CO2 / kg food)    float64
dtype: object

In [63]:
path = os.path.join(os.getcwd(), "data", "processed", "Mapping.csv")
mapping.to_csv(path, index = False, header = True)

#### GHG Factors Calculation for Preps

In [64]:
Preps['GHG Emission/g'] = 0

In [65]:
def get_items_prep_ghgf(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Preps.loc[index,'PrepId']]
    #print(ingres)
    ghgf = Preps.loc[index, 'GHG Emission/g']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            #print(ghge)
            if Preps.loc[index,'PakUOM'] in ['L', 'Kg'] and ingres.loc[idx,'Uom'] in ['L', 'Kg']:
                weight = float(ingres.loc[idx,'InvFactor'])
                #print(weight, ghge)
                ghgf += weight*float(ghge)
            if Preps.loc[index,'PakUOM'] in ['ml', 'g'] and ingres.loc[idx,'Uom'] in ['ml', 'g']:
                weight = float(ingres.loc[idx,'InvFactor'])
                #print(weight, ghge)
                ghgf += weight*float(ghge)
            if Preps.loc[index,'PakUOM'] in ['L', 'Kg'] and ingres.loc[idx,'Uom'] in ['ml', 'g']:
                weight = float(ingres.loc[idx,'InvFactor'])*0.001
                #print(weight, ghge)
                ghgf += weight*float(ghge)
            if Preps.loc[index,'PakUOM'] in ['ml', 'g'] and ingres.loc[idx,'Uom'] in ['L', 'Kg']:
                weight = float(ingres.loc[idx,'InvFactor'])*1000
                #print(weight, ghge)
                ghgf += weight*float(ghge)
    Preps.loc[index, 'GHG Emission/g'] = float(ghgf)

In [66]:
for index, row in Preps.iterrows():
    get_items_prep_ghgf(index , row)

In [67]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup,StdQty,StdUom,GHG Emission/g
0,P-5545,PREP|Bechamel,10.000,L,PREP,10000,ml,2.966011
1,P-30411,PREP|Cooked Spelt,2.000,Kg,,2000,g,0.739250
2,P-5962,PREP|Pesto,20.000,L,PREP,20000,ml,1.207155
3,P-10395,PREP|Sauce Cheese,15.000,L,,15000,ml,1.040140
4,P-30409,PREP|Tri Colour Rotini,2.000,Kg,,2000,g,0.761250
5,P-30410,SALAD| Antipasto Pasta,2050.000,g,,2050.000,g,1.138097
6,P-30412,SALAD|Spelt|Arugula,1200.000,g,,1200.000,g,0.614064
7,P-25843,SALAD|Very Berry Spelt,2000.000,g,,2000.000,g,0.810187
8,P-25844,SALAD|Watermelon tomato,2800.000,g,,2800.000,g,1.281312
9,P-56773,DRAIN|Pepperoncini,1.500,Kg,,1500,g,0.670517


#### Standard Unit Converter

In [68]:
Std_Unit = pd.read_csv(os.path.join(os.getcwd(), "data", "raw", "Standard_Conversions.csv"))
Std_Unit

,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,473.176250,1,pt,473.176250,ml
1,14.787000,1,Tbsp,14.787000,ml
2,4.928900,1,tsp,4.928900,ml
3,946.350000,1,qt,946.350000,ml
4,3785.410000,1,gal,3785.410000,ml
5,236.588125,1,cup,236.588125,ml
6,29.573500,1,fl oz,29.573500,ml
7,1000.000000,1,L,1000.000000,ml
8,355.000000,1,355ML,355.000000,ml
9,450.000000,1,450ML,450.000000,ml


In [69]:
def std_converter(qty, uom):
    if uom in Std_Unit['ConvertFromUom'].tolist():
        multiplier = Std_Unit.loc[Std_Unit['ConvertFromUom'] == uom, 'Multiplier']
        return float(qty)*float(multiplier)
    else:
        return float(qty)

In [70]:
def spc_converter(ingre, qty, uom):
    if ingre in Conversions['ConversionId'].tolist():
        conversion = Conversions.loc[(Conversions['ConversionId'] == ingre) & (Conversions['ConvertFromUom'] == uom)]
        multiplier = conversion['Multiplier']
        if multiplier.empty:
            return std_converter(qty, uom)
        else:
            return float(qty)/float(multiplier)
    else:
        return std_converter(qty, uom)

In [71]:
std_converter(1,'591ML')

591.0

In [72]:
spc_converter('I-4291', 0.33, 'BUNCH')

200.63961477193965

### GHG Calculation for Products

In [73]:
Products['GHG Emission (g)'] = np.nan
Products['Weight (g)'] = np.nan

In [74]:
for index, row in Products.iterrows():
    ghg = 0
    weight = 0
    Products.loc[index, 'GHG Emission (g)'] = ghg
    Products.loc[index, 'Weight (g)'] = ghg

In [75]:
ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[66,'ProdId']]
ingres

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
260,I-17203,15.000,ml,0.00100000,1.0000,R-58300
261,I-29439,250.000,g,0.00100000,1.0000,R-58300
262,I-4291,0.330,BUNCH,1.00000000,1.0000,R-58300
263,I-45558,20.000,g,1.00000000,1.0000,R-58300
264,I-6807,30.000,g,0.00220462,1.0000,R-58300
265,I-9702,3.000,g,0.00100000,1.0000,R-58300
266,P-56729,112.000,g,1.00000000,1.0000,R-58300
267,P-58299,100.000,g,0.00100000,1.0000,R-58300


In [76]:
ghg = 0
for idx, row in ingres.iterrows():
    ingre = ingres.loc[idx,'IngredientId']
    if ingre.startswith('I'):
        ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
        if (ingres.loc[idx,'Uom'] == 'g') or (ingres.loc[idx,'Uom'] == 'ml'):
            qty = float(ingres.loc[idx,'Qty'])
            ghg += qty*float(ghge)   
        print(ghge)
        print(ghg)

503    1.93
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
28.95
163    1.5225
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
409.575
257    0.622
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
409.575
191    3.85686
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
486.7122
289    0.3015
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
495.7572
441    9.3703
Name: Active Total Supply Chain Emissions (kg CO2 / kg food), dtype: float64
523.8681


In [77]:
def get_items_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    #print(ingres)
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('I'):
            ghge = mapping.loc[mapping['ItemId'] == ingre, 'Active Total Supply Chain Emissions (kg CO2 / kg food)']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)
                weight += qty
                ghg += qty*float(ghge)
            #print(ingre, Qty, Uom, qty)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [78]:
def get_preps_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('P'):
            ghge = Preps.loc[Preps['PrepId'] == ingre, 'GHG Emission/g']
            Qty = float(ingres.loc[idx,'Qty'])
            Uom = ingres.loc[idx,'Uom']
            if ingre in Conversions['ConversionId'].tolist():
                qty = spc_converter(ingre, Qty, Uom)
                weight += qty
                ghg += qty*float(ghge)
            else:
                qty = std_converter(Qty, Uom)
                weight += qty
                ghg += qty*float(ghge)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [79]:
def get_products_ghge(index, row):
    ingres = Ingredients.loc[Ingredients['Recipe'] == Products.loc[index,'ProdId']]
    ghg = Products.loc[index, 'GHG Emission (g)']
    weight = Products.loc[index, 'Weight (g)']
    for idx, row in ingres.iterrows():
        ingre = ingres.loc[idx,'IngredientId']
        if ingre.startswith('R'):
            ghge = Products.loc[Products['ProdId'] == ingre, 'GHG Emission (g)']
            Weight = Products.loc[Products['ProdId'] == ingre, 'Weight (g)']
            Qty = float(ingres.loc[idx,'Qty'])
            ghg += Qty*float(ghge)
            weight += Qty*float(Weight)
    Products.loc[index, 'GHG Emission (g)'] = float(ghg)
    Products.loc[index, 'Weight (g)'] = float(weight)

In [80]:
for index, row in Products.iterrows():
    get_items_ghge(index , row)
    get_preps_ghge(index, row)
    get_products_ghge(index, row)

In [83]:
Products['GHG Emission (Kg) / 100g'] = 0.1*Products['GHG Emission (g)']/Products['Weight (g)']

In [84]:
Products

,ProdId,Description,GHG Emission (g),Weight (g),GHG Emission (Kg) / 100g
0,R-54100,BEV|Eggnog Chai,0.000000,1000.000000,0.000000
1,R-50814,BEV|Water|Dasani|591ml,0.000000,1.000000,0.000000
2,R-25866,"MC- Salad Watermelon, tomato",512.524648,400.000000,0.128131
3,R-30415,Salad- Spelt & Arugula,110.531567,211.000000,0.052385
4,R-30414,SALAD|Antipasto Pasta,398.333957,351.000000,0.113485
5,R-10224,SALAD|Caesar,137.202000,175.000000,0.078401
6,R-10222,SALAD|Caprese,803.652049,210.718400,0.381387
7,R-31439,SALAD|Summer Watermelon,512.524648,400.000000,0.128131
8,R-25867,SALAD|Very Berry Spelt,283.565450,350.000000,0.081019
9,R-31440,SALAD|Very Berry/Spelt,283.565450,350.000000,0.081019


In [82]:
path = os.path.join(os.getcwd(), "data", "final", "Recipes GHG.csv")
Products.to_csv(path, index = False, header = True)